# MongoDB Database Creation

According to these two data sets, one climate_historic might contain 0 to many hotspot data sets. This is a 1:M relationship between the climate and the hotspot data. We will use embedded data form, in order to give a better read and scalability performance.


The example shows the design of the data model we have done for the efficient querying of finding oneday's information.  

In [1]:
# Library load
import csv
import datetime
from datetime import datetime
from datetime import timedelta
import pymongo
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Initial the cilent and create db and collection
client = MongoClient() 
db = client.skill_demo_db 
db.drop_collection('vic_fire_collection') 
collection = db.vic_fire_collection

In [3]:
# load data from two files and establish the database model, which is embedded
with open('hotspot_historic.csv') as file:
    hotspot={}
    reader = csv.reader(file)
    next(reader, None) # avoid the header
    for row in reader:
        temp=[]
        if row[4] not in hotspot.keys():# if key not in hotspot, create a new key
            hotspot_data={'Latitude': float(row[0]),
                          'Longitude': float(row[1]),
                          'Datetime' : datetime.strptime(row[2],"%Y-%m-%dT%X"),
                          'Confidence': int(row[3]),
                          'Date': datetime.strptime(row[4],"%d/%m/%Y"),
                          'Surface_Temperature_Celcius': int(row[5])
                           }
            temp.append(hotspot_data)
            hotspot[row[4]]=temp   
        else: #key exists, append to the list.
            hotspot_data={'Latitude': float(row[0]),
                          'Longitude': float(row[1]),
                          'Datetime' : datetime.strptime(row[2],"%Y-%m-%dT%X"),
                          'Confidence': int(row[3]),
                          'Date': datetime.strptime(row[4],"%d/%m/%Y"),
                          'Surface_Temperature_Celcius': int(row[5])
                           }
            temp=hotspot[row[4]] #update existed key
            temp.append(hotspot_data) # append new hotspot 
            hotspot[row[4]]=temp
            

with open('climate_historic.csv') as file:
    climate={}
    reader = csv.reader(file)
    next(reader, None) # avoid the header 
    for row in reader:
        if row[1] in hotspot.keys(): 
            ahotspot=hotspot[row[1]]
            one_day={'Station' :int(row[0]),
                     'Date': datetime.strptime(row[1],"%d/%m/%Y"),
                     'Air_Temperature_Celcius' : int(row[2]),
                     'Relative_Humidity' : float(row[3]),
                     'WindSpeed_Knots' : float(row[4]),
                     'Max_Wind_Speed' : float(row[5]),
                     'Precipitation' : row[6],
                     'Hotspot' : ahotspot # store hotspot as a attribute inside the climate, so embedded.
                      }     
        else: # Store climate without hotspot
            one_day={'Station' : int(row[0]),
                     'Date': datetime.strptime(row[1],"%d/%m/%Y"),
                     'Air_Temperature_Celcius' : int(row[2]),
                     'Relative_Humidity' : float(row[3]),
                     'WindSpeed_Knots' : float(row[4]),
                     'Max_Wind_Speed' : float(row[5]),
                     'Precipitation' : row[6],
                     'Hotspot' : []
                      }
        collection.insert_one(one_day) 

In [4]:
# take a look 
cursor=collection.find({})
for document in cursor:
    pprint(document)

{'Air_Temperature_Celcius': 19,
 'Date': datetime.datetime(2017, 12, 31, 0, 0),
 'Hotspot': [],
 'Max_Wind_Speed': 11.1,
 'Precipitation': ' 0.00I',
 'Relative_Humidity': 56.8,
 'Station': 948700,
 'WindSpeed_Knots': 7.9,
 '_id': ObjectId('5d99984ffeb25d7bde46c0c8')}
{'Air_Temperature_Celcius': 15,
 'Date': datetime.datetime(2018, 1, 2, 0, 0),
 'Hotspot': [],
 'Max_Wind_Speed': 13.0,
 'Precipitation': ' 0.02G',
 'Relative_Humidity': 50.7,
 'Station': 948700,
 'WindSpeed_Knots': 9.2,
 '_id': ObjectId('5d99984ffeb25d7bde46c0c9')}
{'Air_Temperature_Celcius': 16,
 'Date': datetime.datetime(2018, 1, 3, 0, 0),
 'Hotspot': [],
 'Max_Wind_Speed': 15.0,
 'Precipitation': ' 0.00G',
 'Relative_Humidity': 53.6,
 'Station': 948700,
 'WindSpeed_Knots': 8.1,
 '_id': ObjectId('5d99984ffeb25d7bde46c0ca')}
{'Air_Temperature_Celcius': 24,
 'Date': datetime.datetime(2018, 1, 4, 0, 0),
 'Hotspot': [],
 'Max_Wind_Speed': 14.0,
 'Precipitation': ' 0.00I',
 'Relative_Humidity': 61.6,
 'Station': 948700,
 'Win

 '_id': ObjectId('5d99984ffeb25d7bde46c0ef')}
{'Air_Temperature_Celcius': 21,
 'Date': datetime.datetime(2018, 2, 10, 0, 0),
 'Hotspot': [],
 'Max_Wind_Speed': 12.0,
 'Precipitation': ' 0.00I',
 'Relative_Humidity': 60.4,
 'Station': 948700,
 'WindSpeed_Knots': 6.8,
 '_id': ObjectId('5d99984ffeb25d7bde46c0f0')}
{'Air_Temperature_Celcius': 19,
 'Date': datetime.datetime(2018, 2, 11, 0, 0),
 'Hotspot': [],
 'Max_Wind_Speed': 15.0,
 'Precipitation': ' 0.02G',
 'Relative_Humidity': 53.8,
 'Station': 948700,
 'WindSpeed_Knots': 9.2,
 '_id': ObjectId('5d99984ffeb25d7bde46c0f1')}
{'Air_Temperature_Celcius': 16,
 'Date': datetime.datetime(2018, 2, 12, 0, 0),
 'Hotspot': [],
 'Max_Wind_Speed': 19.0,
 'Precipitation': ' 0.00G',
 'Relative_Humidity': 48.3,
 'Station': 948700,
 'WindSpeed_Knots': 14.4,
 '_id': ObjectId('5d99984ffeb25d7bde46c0f2')}
{'Air_Temperature_Celcius': 16,
 'Date': datetime.datetime(2018, 2, 13, 0, 0),
 'Hotspot': [],
 'Max_Wind_Speed': 16.9,
 'Precipitation': ' 0.16G',
 'Re

              'Surface_Temperature_Celcius': 42},
             {'Confidence': 54,
              'Date': datetime.datetime(2018, 3, 10, 0, 0),
              'Datetime': datetime.datetime(2018, 3, 10, 4, 42, 30),
              'Latitude': -37.2197,
              'Longitude': 147.9621,
              'Surface_Temperature_Celcius': 43},
             {'Confidence': 74,
              'Date': datetime.datetime(2018, 3, 10, 0, 0),
              'Datetime': datetime.datetime(2018, 3, 10, 4, 39),
              'Latitude': -37.2128,
              'Longitude': 147.9308,
              'Surface_Temperature_Celcius': 48}],
 'Max_Wind_Speed': 12.0,
 'Precipitation': ' 0.00I',
 'Relative_Humidity': 60.2,
 'Station': 948701,
 'WindSpeed_Knots': 6.4,
 '_id': ObjectId('5d999850feb25d7bde46c10c')}
{'Air_Temperature_Celcius': 20,
 'Date': datetime.datetime(2018, 3, 11, 0, 0),
 'Hotspot': [],
 'Max_Wind_Speed': 15.9,
 'Precipitation': ' 0.01G',
 'Relative_Humidity': 59.7,
 'Station': 948701,
 'WindSpeed_Knots

              'Datetime': datetime.datetime(2018, 3, 19, 4, 36),
              'Latitude': -37.3982,
              'Longitude': 147.0001,
              'Surface_Temperature_Celcius': 58},
             {'Confidence': 90,
              'Date': datetime.datetime(2018, 3, 19, 0, 0),
              'Datetime': datetime.datetime(2018, 3, 19, 4, 36),
              'Latitude': -37.4523,
              'Longitude': 147.0175,
              'Surface_Temperature_Celcius': 66},
             {'Confidence': 59,
              'Date': datetime.datetime(2018, 3, 19, 0, 0),
              'Datetime': datetime.datetime(2018, 3, 19, 4, 35),
              'Latitude': -37.4023,
              'Longitude': 147.0394,
              'Surface_Temperature_Celcius': 56},
             {'Confidence': 60,
              'Date': datetime.datetime(2018, 3, 19, 0, 0),
              'Datetime': datetime.datetime(2018, 3, 19, 4, 33, 50),
              'Latitude': -37.4336,
              'Longitude': 147.0534,
              'Sur

 'Hotspot': [{'Confidence': 88,
              'Date': datetime.datetime(2018, 3, 28, 0, 0),
              'Datetime': datetime.datetime(2018, 3, 28, 4, 38),
              'Latitude': -36.4721,
              'Longitude': 142.2586,
              'Surface_Temperature_Celcius': 64},
             {'Confidence': 72,
              'Date': datetime.datetime(2018, 3, 28, 0, 0),
              'Datetime': datetime.datetime(2018, 3, 28, 4, 37, 30),
              'Latitude': -36.7241,
              'Longitude': 141.0073,
              'Surface_Temperature_Celcius': 46},
             {'Confidence': 86,
              'Date': datetime.datetime(2018, 3, 28, 0, 0),
              'Datetime': datetime.datetime(2018, 3, 28, 4, 33, 40),
              'Latitude': -38.004,
              'Longitude': 143.9249,
              'Surface_Temperature_Celcius': 60},
             {'Confidence': 92,
              'Date': datetime.datetime(2018, 3, 28, 0, 0),
              'Datetime': datetime.datetime(2018, 3, 28, 4, 3

...

 'Hotspot': [{'Confidence': 77,
              'Date': datetime.datetime(2018, 10, 26, 0, 0),
              'Datetime': datetime.datetime(2018, 10, 26, 13, 30, 31),
              'Latitude': -35.325,
              'Longitude': 143.497,
              'Surface_Temperature_Celcius': 36},
             {'Confidence': 100,
              'Date': datetime.datetime(2018, 10, 26, 0, 0),
              'Datetime': datetime.datetime(2018, 10, 26, 13, 30, 31),
              'Latitude': -35.321,
              'Longitude': 143.502,
              'Surface_Temperature_Celcius': 48},
             {'Confidence': 100,
              'Date': datetime.datetime(2018, 10, 26, 0, 0),
              'Datetime': datetime.datetime(2018, 10, 26, 13, 30, 31),
              'Latitude': -35.323,
              'Longitude': 143.509,
              'Surface_Temperature_Celcius': 53},
             {'Confidence': 67,
              'Date': datetime.datetime(2018, 10, 26, 0, 0),
              'Datetime': datetime.datetime(2018, 

              'Latitude': -37.864,
              'Longitude': 144.174,
              'Surface_Temperature_Celcius': 41},
             {'Confidence': 56,
              'Date': datetime.datetime(2018, 11, 12, 0, 0),
              'Datetime': datetime.datetime(2018, 11, 12, 4, 46, 53),
              'Latitude': -38.912,
              'Longitude': 146.272,
              'Surface_Temperature_Celcius': 48},
             {'Confidence': 67,
              'Date': datetime.datetime(2018, 11, 12, 0, 0),
              'Datetime': datetime.datetime(2018, 11, 12, 0, 33, 15),
              'Latitude': -36.942,
              'Longitude': 143.292,
              'Surface_Temperature_Celcius': 54},
             {'Confidence': 89,
              'Date': datetime.datetime(2018, 11, 12, 0, 0),
              'Datetime': datetime.datetime(2018, 11, 12, 0, 33, 15),
              'Latitude': -36.94,
              'Longitude': 143.281,
              'Surface_Temperature_Celcius': 69}],
 'Max_Wind_Speed': 14.0,
 '

              'Date': datetime.datetime(2018, 11, 29, 0, 0),
              'Datetime': datetime.datetime(2018, 11, 29, 3, 52, 10),
              'Latitude': -37.605,
              'Longitude': 149.326,
              'Surface_Temperature_Celcius': 44},
             {'Confidence': 97,
              'Date': datetime.datetime(2018, 11, 29, 0, 0),
              'Datetime': datetime.datetime(2018, 11, 29, 3, 52, 10),
              'Latitude': -37.61,
              'Longitude': 149.307,
              'Surface_Temperature_Celcius': 80},
             {'Confidence': 99,
              'Date': datetime.datetime(2018, 11, 29, 0, 0),
              'Datetime': datetime.datetime(2018, 11, 29, 3, 52, 10),
              'Latitude': -37.609,
              'Longitude': 149.32,
              'Surface_Temperature_Celcius': 84}],
 'Max_Wind_Speed': 16.9,
 'Precipitation': ' 0.00I',
 'Relative_Humidity': 56.7,
 'Station': 948702,
 'WindSpeed_Knots': 9.3,
 '_id': ObjectId('5d999850feb25d7bde46c214')}
{'Air_Tem

              'Date': datetime.datetime(2018, 12, 12, 0, 0),
              'Datetime': datetime.datetime(2018, 12, 12, 0, 45, 38),
              'Latitude': -37.903,
              'Longitude': 145.25,
              'Surface_Temperature_Celcius': 44}],
 'Max_Wind_Speed': 12.0,
 'Precipitation': ' 0.00I',
 'Relative_Humidity': 55.3,
 'Station': 948702,
 'WindSpeed_Knots': 6.2,
 '_id': ObjectId('5d999850feb25d7bde46c221')}
{'Air_Temperature_Celcius': 28,
 'Date': datetime.datetime(2018, 12, 13, 0, 0),
 'Hotspot': [{'Confidence': 83,
              'Date': datetime.datetime(2018, 12, 13, 0, 0),
              'Datetime': datetime.datetime(2018, 12, 13, 4, 4, 19),
              'Latitude': -38.231,
              'Longitude': 147.172,
              'Surface_Temperature_Celcius': 60}],
 'Max_Wind_Speed': 19.0,
 'Precipitation': ' 0.00I',
 'Relative_Humidity': 49.9,
 'Station': 948702,
 'WindSpeed_Knots': 11.0,
 '_id': ObjectId('5d999850feb25d7bde46c222')}
{'Air_Temperature_Celcius': 17,
 'Date'

 'Station': 948702,
 'WindSpeed_Knots': 8.3,
 '_id': ObjectId('5d999850feb25d7bde46c22c')}
{'Air_Temperature_Celcius': 16,
 'Date': datetime.datetime(2018, 12, 24, 0, 0),
 'Hotspot': [{'Confidence': 65,
              'Date': datetime.datetime(2018, 12, 24, 0, 0),
              'Datetime': datetime.datetime(2018, 12, 24, 13, 12, 1),
              'Latitude': -35.646,
              'Longitude': 142.282,
              'Surface_Temperature_Celcius': 32}],
 'Max_Wind_Speed': 16.9,
 'Precipitation': ' 0.00I',
 'Relative_Humidity': 47.0,
 'Station': 948702,
 'WindSpeed_Knots': 12.0,
 '_id': ObjectId('5d999850feb25d7bde46c22d')}
{'Air_Temperature_Celcius': 17,
 'Date': datetime.datetime(2018, 12, 25, 0, 0),
 'Hotspot': [{'Confidence': 80,
              'Date': datetime.datetime(2018, 12, 25, 0, 0),
              'Datetime': datetime.datetime(2018, 12, 25, 4, 29, 8),
              'Latitude': -37.708,
              'Longitude': 145.1,
              'Surface_Temperature_Celcius': 54}],
 'Max_Win

# **Database manipulation examples. You can skip it if you only need the database been created.**

**a.Find climate data on 10th December 2018.**

In [5]:
date_a = datetime.strptime('10/12/2018',"%d/%m/%Y")

search_A = collection.find({'Date' : date_a},{"Hotspot":0 , "_id":0}) 
for documents in search_A:
    pprint(documents)

{'Air_Temperature_Celcius': 17,
 'Date': datetime.datetime(2018, 12, 10, 0, 0),
 'Max_Wind_Speed': 14.0,
 'Precipitation': ' 0.00I',
 'Relative_Humidity': 53.5,
 'Station': 948702,
 'WindSpeed_Knots': 7.3}


**b.Find the latitude, longitude, surface temperature (°C), and confidence when
the surface temperature (°C) was between 65 °C and 100 °C.**

In [6]:
search_B = collection.aggregate(
    [{"$unwind": "$Hotspot"},      
           {"$match" : {
                         "Hotspot.Surface_Temperature_Celcius" : { "$gte":65 , "$lte":100}
                       }
           }, 
           {"$project" : { "_id":0, 
                           "Hotspot.Latitude":1, 
                           "Hotspot.Longitude":1 , 
                           "Hotspot.Confidence":1,
                           "Hotspot.Surface_Temperature_Celcius":1
                          }
           }
    ])


for documents in search_B:
    pprint(documents)   

{'Hotspot': {'Confidence': 94,
             'Latitude': -37.2284,
             'Longitude': 147.9187,
             'Surface_Temperature_Celcius': 73}}
{'Hotspot': {'Confidence': 97,
             'Latitude': -37.6572,
             'Longitude': 142.0703,
             'Surface_Temperature_Celcius': 80}}
{'Hotspot': {'Confidence': 84,
             'Latitude': -37.0193,
             'Longitude': 148.1459,
             'Surface_Temperature_Celcius': 71}}
{'Hotspot': {'Confidence': 100,
             'Latitude': -37.4229,
             'Longitude': 147.027,
             'Surface_Temperature_Celcius': 99}}
{'Hotspot': {'Confidence': 80,
             'Latitude': -37.0055,
             'Longitude': 148.1582,
             'Surface_Temperature_Celcius': 68}}
{'Hotspot': {'Confidence': 85,
             'Latitude': -37.4128,
             'Longitude': 147.0242,
             'Surface_Temperature_Celcius': 98}}
{'Hotspot': {'Confidence': 90,
             'Latitude': -34.357,
             'Longitude': 141

             'Surface_Temperature_Celcius': 92}}
{'Hotspot': {'Confidence': 100,
             'Latitude': -36.1819,
             'Longitude': 145.9269,
             'Surface_Temperature_Celcius': 98}}
{'Hotspot': {'Confidence': 87,
             'Latitude': -37.6387,
             'Longitude': 142.9032,
             'Surface_Temperature_Celcius': 88}}
{'Hotspot': {'Confidence': 93,
             'Latitude': -37.6439,
             'Longitude': 142.913,
             'Surface_Temperature_Celcius': 83}}
{'Hotspot': {'Confidence': 93,
             'Latitude': -37.3165,
             'Longitude': 147.4932,
             'Surface_Temperature_Celcius': 72}}
{'Hotspot': {'Confidence': 95,
             'Latitude': -37.3158,
             'Longitude': 147.4986,
             'Surface_Temperature_Celcius': 76}}
{'Hotspot': {'Confidence': 89,
             'Latitude': -37.4471,
             'Longitude': 147.6331,
             'Surface_Temperature_Celcius': 65}}
{'Hotspot': {'Confidence': 90,
             '

{'Hotspot': {'Confidence': 96,
             'Latitude': -36.9498,
             'Longitude': 142.2012,
             'Surface_Temperature_Celcius': 78}}
{'Hotspot': {'Confidence': 93,
             'Latitude': -36.1941,
             'Longitude': 143.1604,
             'Surface_Temperature_Celcius': 72}}
{'Hotspot': {'Confidence': 100,
             'Latitude': -36.4142,
             'Longitude': 143.1077,
             'Surface_Temperature_Celcius': 97}}
{'Hotspot': {'Confidence': 92,
             'Latitude': -36.1924,
             'Longitude': 143.1716,
             'Surface_Temperature_Celcius': 70}}
{'Hotspot': {'Confidence': 89,
             'Latitude': -36.0295,
             'Longitude': 143.6409,
             'Surface_Temperature_Celcius': 65}}
{'Hotspot': {'Confidence': 100,
             'Latitude': -37.0761,
             'Longitude': 141.0574,
             'Surface_Temperature_Celcius': 89}}
{'Hotspot': {'Confidence': 93,
             'Latitude': -35.0616,
             'Longitude': 

{'Hotspot': {'Confidence': 100,
             'Latitude': -35.1857,
             'Longitude': 141.0604,
             'Surface_Temperature_Celcius': 92}}
{'Hotspot': {'Confidence': 100,
             'Latitude': -35.6469,
             'Longitude': 142.376,
             'Surface_Temperature_Celcius': 93}}
{'Hotspot': {'Confidence': 91,
             'Latitude': -35.6453,
             'Longitude': 142.387,
             'Surface_Temperature_Celcius': 69}}
{'Hotspot': {'Confidence': 100,
             'Latitude': -36.4409,
             'Longitude': 141.6948,
             'Surface_Temperature_Celcius': 92}}
{'Hotspot': {'Confidence': 100,
             'Latitude': -37.9174,
             'Longitude': 145.8678,
             'Surface_Temperature_Celcius': 93}}
{'Hotspot': {'Confidence': 95,
             'Latitude': -37.9152,
             'Longitude': 145.8808,
             'Surface_Temperature_Celcius': 77}}
{'Hotspot': {'Confidence': 95,
             'Latitude': -37.8906,
             'Longitude': 

{'Hotspot': {'Confidence': 98,
             'Latitude': -36.5262,
             'Longitude': 144.5737,
             'Surface_Temperature_Celcius': 84}}
{'Hotspot': {'Confidence': 95,
             'Latitude': -36.6995,
             'Longitude': 142.978,
             'Surface_Temperature_Celcius': 76}}
{'Hotspot': {'Confidence': 93,
             'Latitude': -36.8879,
             'Longitude': 141.6647,
             'Surface_Temperature_Celcius': 72}}
{'Hotspot': {'Confidence': 100,
             'Latitude': -35.8768,
             'Longitude': 145.4627,
             'Surface_Temperature_Celcius': 86}}
{'Hotspot': {'Confidence': 93,
             'Latitude': -36.4899,
             'Longitude': 142.2805,
             'Surface_Temperature_Celcius': 73}}
{'Hotspot': {'Confidence': 93,
             'Latitude': -36.663,
             'Longitude': 141.3038,
             'Surface_Temperature_Celcius': 72}}
{'Hotspot': {'Confidence': 94,
             'Latitude': -36.092,
             'Longitude': 141.

             'Latitude': -36.7751,
             'Longitude': 145.1696,
             'Surface_Temperature_Celcius': 68}}
{'Hotspot': {'Confidence': 91,
             'Latitude': -36.3961,
             'Longitude': 140.9781,
             'Surface_Temperature_Celcius': 69}}
{'Hotspot': {'Confidence': 100,
             'Latitude': -36.7708,
             'Longitude': 145.1908,
             'Surface_Temperature_Celcius': 98}}
{'Hotspot': {'Confidence': 98,
             'Latitude': -37.758,
             'Longitude': 143.179,
             'Surface_Temperature_Celcius': 82}}
{'Hotspot': {'Confidence': 94,
             'Latitude': -37.7609,
             'Longitude': 143.1633,
             'Surface_Temperature_Celcius': 74}}
{'Hotspot': {'Confidence': 89,
             'Latitude': -36.5116,
             'Longitude': 144.562,
             'Surface_Temperature_Celcius': 66}}
{'Hotspot': {'Confidence': 92,
             'Latitude': -36.803,
             'Longitude': 142.9261,
             'Surface_Temp

{'Hotspot': {'Confidence': 90,
             'Latitude': -36.7032,
             'Longitude': 144.0992,
             'Surface_Temperature_Celcius': 66}}
{'Hotspot': {'Confidence': 96,
             'Latitude': -36.3402,
             'Longitude': 141.7429,
             'Surface_Temperature_Celcius': 77}}
{'Hotspot': {'Confidence': 89,
             'Latitude': -36.5661,
             'Longitude': 142.2956,
             'Surface_Temperature_Celcius': 65}}
{'Hotspot': {'Confidence': 85,
             'Latitude': -36.5023,
             'Longitude': 143.6038,
             'Surface_Temperature_Celcius': 66}}
{'Hotspot': {'Confidence': 90,
             'Latitude': -36.8616,
             'Longitude': 142.8025,
             'Surface_Temperature_Celcius': 78}}
{'Hotspot': {'Confidence': 90,
             'Latitude': -38.0372,
             'Longitude': 146.0616,
             'Surface_Temperature_Celcius': 67}}
{'Hotspot': {'Confidence': 94,
             'Latitude': -38.049,
             'Longitude': 146

             'Longitude': 146.157,
             'Surface_Temperature_Celcius': 71}}
{'Hotspot': {'Confidence': 92,
             'Latitude': -37.95,
             'Longitude': 142.366,
             'Surface_Temperature_Celcius': 70}}
{'Hotspot': {'Confidence': 93,
             'Latitude': -37.875,
             'Longitude': 142.51,
             'Surface_Temperature_Celcius': 73}}
{'Hotspot': {'Confidence': 95,
             'Latitude': -37.613,
             'Longitude': 149.305,
             'Surface_Temperature_Celcius': 75}}
{'Hotspot': {'Confidence': 90,
             'Latitude': -37.624,
             'Longitude': 149.314,
             'Surface_Temperature_Celcius': 66}}
{'Hotspot': {'Confidence': 93,
             'Latitude': -38.057,
             'Longitude': 144.211,
             'Surface_Temperature_Celcius': 73}}
{'Hotspot': {'Confidence': 78,
             'Latitude': -37.966,
             'Longitude': 145.051,
             'Surface_Temperature_Celcius': 68}}
{'Hotspot': {'Confidence

**c.Find date, surface temperature (°C), air temperature (°C), relative humidity and
max wind speed on 15th and 16th of December 2018.**

In [7]:
date_c1 = datetime.strptime("15/12/2018","%d/%m/%Y")
date_c2 = datetime.strptime("16/12/2018","%d/%m/%Y")

search_C = collection.find( 
{"Date" : { "$in":[date_c1 , date_c2]}},
{
    "_id":0 , 
    "Date":1 ,
    "Hotspot.Surface_Temperature_Celcius":1 , 
    "Air_Temperature_Celcius":1 ,
    "Relative_Humidity":1 ,
    "Max_Wind_Speed":1  
}  
)
                    
for documents in search_C:
    pprint(documents)

{'Air_Temperature_Celcius': 18,
 'Date': datetime.datetime(2018, 12, 15, 0, 0),
 'Hotspot': [{'Surface_Temperature_Celcius': 42},
             {'Surface_Temperature_Celcius': 36},
             {'Surface_Temperature_Celcius': 38},
             {'Surface_Temperature_Celcius': 40}],
 'Max_Wind_Speed': 14.0,
 'Relative_Humidity': 52.0}
{'Air_Temperature_Celcius': 18,
 'Date': datetime.datetime(2018, 12, 16, 0, 0),
 'Hotspot': [{'Surface_Temperature_Celcius': 43},
             {'Surface_Temperature_Celcius': 33},
             {'Surface_Temperature_Celcius': 54},
             {'Surface_Temperature_Celcius': 73},
             {'Surface_Temperature_Celcius': 55},
             {'Surface_Temperature_Celcius': 75},
             {'Surface_Temperature_Celcius': 55},
             {'Surface_Temperature_Celcius': 66},
             {'Surface_Temperature_Celcius': 56},
             {'Surface_Temperature_Celcius': 60},
             {'Surface_Temperature_Celcius': 73},
             {'Surface_Temperature_C

**d.Find datetime, air temperature (°C), surface temperature (°C) and confidence
when the confidence is between 80 and 100.**

In [8]:
search_D = collection.aggregate([    
          {"$unwind": "$Hotspot"},      
          {"$match" : {"Hotspot.Confidence" : { "$gte":80 , "$lte":100}}}, 
          {"$project" : 
                    {
                     "_id":0,  
                     "Hotspot.Surface_Temperature_Celcius":1 ,
                     "Hotspot.Datetime" : 1,
                     "Hotspot.Confidence" :1,
                     "Air_Temperature_Celcius":1
                    } 
          } 
          ])

for documents in search_D:
    pprint(documents)

{'Air_Temperature_Celcius': 20,
 'Hotspot': {'Confidence': 87,
             'Datetime': datetime.datetime(2018, 3, 6, 5, 6, 30),
             'Surface_Temperature_Celcius': 62}}
{'Air_Temperature_Celcius': 20,
 'Hotspot': {'Confidence': 85,
             'Datetime': datetime.datetime(2018, 3, 6, 5, 6, 20),
             'Surface_Temperature_Celcius': 59}}
{'Air_Temperature_Celcius': 19,
 'Hotspot': {'Confidence': 88,
             'Datetime': datetime.datetime(2018, 3, 7, 4, 16, 10),
             'Surface_Temperature_Celcius': 64}}
{'Air_Temperature_Celcius': 23,
 'Hotspot': {'Confidence': 86,
             'Datetime': datetime.datetime(2018, 3, 9, 13, 23, 40),
             'Surface_Temperature_Celcius': 41}}
{'Air_Temperature_Celcius': 19,
 'Hotspot': {'Confidence': 100,
             'Datetime': datetime.datetime(2018, 3, 10, 4, 48, 40),
             'Surface_Temperature_Celcius': 105}}
{'Air_Temperature_Celcius': 19,
 'Hotspot': {'Confidence': 100,
             'Datetime': datetime.datet

{'Air_Temperature_Celcius': 18,
 'Hotspot': {'Confidence': 97,
             'Datetime': datetime.datetime(2018, 3, 28, 4, 27, 40),
             'Surface_Temperature_Celcius': 80}}
{'Air_Temperature_Celcius': 18,
 'Hotspot': {'Confidence': 100,
             'Datetime': datetime.datetime(2018, 3, 28, 4, 27, 20),
             'Surface_Temperature_Celcius': 107}}
{'Air_Temperature_Celcius': 18,
 'Hotspot': {'Confidence': 100,
             'Datetime': datetime.datetime(2018, 3, 28, 4, 27, 10),
             'Surface_Temperature_Celcius': 108}}
{'Air_Temperature_Celcius': 18,
 'Hotspot': {'Confidence': 82,
             'Datetime': datetime.datetime(2018, 3, 28, 4, 27),
             'Surface_Temperature_Celcius': 55}}
{'Air_Temperature_Celcius': 18,
 'Hotspot': {'Confidence': 99,
             'Datetime': datetime.datetime(2018, 3, 28, 4, 27),
             'Surface_Temperature_Celcius': 86}}
{'Air_Temperature_Celcius': 18,
 'Hotspot': {'Confidence': 81,
             'Datetime': datetime.datetim

{'Air_Temperature_Celcius': 15,
 'Hotspot': {'Confidence': 93,
             'Datetime': datetime.datetime(2018, 4, 3, 3, 50, 40),
             'Surface_Temperature_Celcius': 72}}
{'Air_Temperature_Celcius': 15,
 'Hotspot': {'Confidence': 95,
             'Datetime': datetime.datetime(2018, 4, 3, 3, 50, 40),
             'Surface_Temperature_Celcius': 76}}
{'Air_Temperature_Celcius': 15,
 'Hotspot': {'Confidence': 86,
             'Datetime': datetime.datetime(2018, 4, 3, 3, 50, 40),
             'Surface_Temperature_Celcius': 60}}
{'Air_Temperature_Celcius': 15,
 'Hotspot': {'Confidence': 89,
             'Datetime': datetime.datetime(2018, 4, 3, 3, 50, 40),
             'Surface_Temperature_Celcius': 65}}
{'Air_Temperature_Celcius': 15,
 'Hotspot': {'Confidence': 82,
             'Datetime': datetime.datetime(2018, 4, 3, 3, 50, 40),
             'Surface_Temperature_Celcius': 55}}
{'Air_Temperature_Celcius': 15,
 'Hotspot': {'Confidence': 89,
             'Datetime': datetime.datetime

             'Surface_Temperature_Celcius': 53}}
{'Air_Temperature_Celcius': 17,
 'Hotspot': {'Confidence': 100,
             'Datetime': datetime.datetime(2018, 4, 5, 13, 9),
             'Surface_Temperature_Celcius': 58}}
{'Air_Temperature_Celcius': 17,
 'Hotspot': {'Confidence': 81,
             'Datetime': datetime.datetime(2018, 4, 5, 13, 9),
             'Surface_Temperature_Celcius': 39}}
{'Air_Temperature_Celcius': 17,
 'Hotspot': {'Confidence': 93,
             'Datetime': datetime.datetime(2018, 4, 5, 13, 8, 40),
             'Surface_Temperature_Celcius': 44}}
{'Air_Temperature_Celcius': 17,
 'Hotspot': {'Confidence': 100,
             'Datetime': datetime.datetime(2018, 4, 5, 13, 8, 40),
             'Surface_Temperature_Celcius': 47}}
{'Air_Temperature_Celcius': 17,
 'Hotspot': {'Confidence': 82,
             'Datetime': datetime.datetime(2018, 4, 5, 13, 7),
             'Surface_Temperature_Celcius': 40}}
{'Air_Temperature_Celcius': 17,
 'Hotspot': {'Confidence': 100,
  

{'Air_Temperature_Celcius': 21,
 'Hotspot': {'Confidence': 97,
             'Datetime': datetime.datetime(2018, 4, 7, 12, 51, 20),
             'Surface_Temperature_Celcius': 45}}
{'Air_Temperature_Celcius': 21,
 'Hotspot': {'Confidence': 100,
             'Datetime': datetime.datetime(2018, 4, 7, 12, 51),
             'Surface_Temperature_Celcius': 49}}
{'Air_Temperature_Celcius': 21,
 'Hotspot': {'Confidence': 92,
             'Datetime': datetime.datetime(2018, 4, 7, 12, 50, 50),
             'Surface_Temperature_Celcius': 43}}
{'Air_Temperature_Celcius': 21,
 'Hotspot': {'Confidence': 89,
             'Datetime': datetime.datetime(2018, 4, 7, 12, 50, 40),
             'Surface_Temperature_Celcius': 42}}
{'Air_Temperature_Celcius': 21,
 'Hotspot': {'Confidence': 80,
             'Datetime': datetime.datetime(2018, 4, 7, 5, 9, 50),
             'Surface_Temperature_Celcius': 54}}
{'Air_Temperature_Celcius': 21,
 'Hotspot': {'Confidence': 80,
             'Datetime': datetime.datetime

...

{'Air_Temperature_Celcius': 12,
 'Hotspot': {'Confidence': 96,
             'Datetime': datetime.datetime(2018, 5, 6, 4, 32, 50),
             'Surface_Temperature_Celcius': 78}}
{'Air_Temperature_Celcius': 12,
 'Hotspot': {'Confidence': 85,
             'Datetime': datetime.datetime(2018, 5, 6, 4, 32, 30),
             'Surface_Temperature_Celcius': 59}}
{'Air_Temperature_Celcius': 12,
 'Hotspot': {'Confidence': 98,
             'Datetime': datetime.datetime(2018, 5, 6, 4, 32, 20),
             'Surface_Temperature_Celcius': 82}}
{'Air_Temperature_Celcius': 12,
 'Hotspot': {'Confidence': 100,
             'Datetime': datetime.datetime(2018, 5, 6, 4, 32, 20),
             'Surface_Temperature_Celcius': 90}}
{'Air_Temperature_Celcius': 12,
 'Hotspot': {'Confidence': 83,
             'Datetime': datetime.datetime(2018, 5, 6, 0, 16, 40),
             'Surface_Temperature_Celcius': 57}}
{'Air_Temperature_Celcius': 12,
 'Hotspot': {'Confidence': 92,
             'Datetime': datetime.datetim

 'Hotspot': {'Confidence': 91,
             'Datetime': datetime.datetime(2018, 5, 11, 4, 50, 40),
             'Surface_Temperature_Celcius': 69}}
{'Air_Temperature_Celcius': 9,
 'Hotspot': {'Confidence': 97,
             'Datetime': datetime.datetime(2018, 5, 11, 4, 50, 40),
             'Surface_Temperature_Celcius': 81}}
{'Air_Temperature_Celcius': 9,
 'Hotspot': {'Confidence': 80,
             'Datetime': datetime.datetime(2018, 5, 11, 4, 50, 40),
             'Surface_Temperature_Celcius': 53}}
{'Air_Temperature_Celcius': 14,
 'Hotspot': {'Confidence': 83,
             'Datetime': datetime.datetime(2018, 5, 12, 3, 57, 40),
             'Surface_Temperature_Celcius': 57}}
{'Air_Temperature_Celcius': 14,
 'Hotspot': {'Confidence': 93,
             'Datetime': datetime.datetime(2018, 5, 12, 3, 56, 10),
             'Surface_Temperature_Celcius': 73}}
{'Air_Temperature_Celcius': 14,
 'Hotspot': {'Confidence': 87,
             'Datetime': datetime.datetime(2018, 5, 12, 3, 56, 10),
   

 'Hotspot': {'Confidence': 90,
             'Datetime': datetime.datetime(2018, 5, 15, 4, 26, 20),
             'Surface_Temperature_Celcius': 67}}
{'Air_Temperature_Celcius': 10,
 'Hotspot': {'Confidence': 94,
             'Datetime': datetime.datetime(2018, 5, 15, 4, 26, 20),
             'Surface_Temperature_Celcius': 74}}
{'Air_Temperature_Celcius': 10,
 'Hotspot': {'Confidence': 100,
             'Datetime': datetime.datetime(2018, 5, 15, 4, 26, 20),
             'Surface_Temperature_Celcius': 110}}
{'Air_Temperature_Celcius': 10,
 'Hotspot': {'Confidence': 89,
             'Datetime': datetime.datetime(2018, 5, 15, 4, 26, 20),
             'Surface_Temperature_Celcius': 65}}
{'Air_Temperature_Celcius': 10,
 'Hotspot': {'Confidence': 95,
             'Datetime': datetime.datetime(2018, 5, 15, 4, 26, 20),
             'Surface_Temperature_Celcius': 76}}
{'Air_Temperature_Celcius': 10,
 'Hotspot': {'Confidence': 81,
             'Datetime': datetime.datetime(2018, 5, 15, 4, 26, 20),

 'Hotspot': {'Confidence': 99,
             'Datetime': datetime.datetime(2018, 9, 24, 15, 7, 47),
             'Surface_Temperature_Celcius': 61}}
{'Air_Temperature_Celcius': 14,
 'Hotspot': {'Confidence': 94,
             'Datetime': datetime.datetime(2018, 9, 24, 15, 7, 45),
             'Surface_Temperature_Celcius': 43}}
{'Air_Temperature_Celcius': 14,
 'Hotspot': {'Confidence': 100,
             'Datetime': datetime.datetime(2018, 9, 24, 15, 7, 45),
             'Surface_Temperature_Celcius': 61}}
{'Air_Temperature_Celcius': 14,
 'Hotspot': {'Confidence': 100,
             'Datetime': datetime.datetime(2018, 9, 24, 13, 30, 10),
             'Surface_Temperature_Celcius': 47}}
{'Air_Temperature_Celcius': 14,
 'Hotspot': {'Confidence': 82,
             'Datetime': datetime.datetime(2018, 9, 24, 13, 30, 9),
             'Surface_Temperature_Celcius': 38}}
{'Air_Temperature_Celcius': 14,
 'Hotspot': {'Confidence': 100,
             'Datetime': datetime.datetime(2018, 9, 24, 13, 30, 9

             'Surface_Temperature_Celcius': 56}}
{'Air_Temperature_Celcius': 26,
 'Hotspot': {'Confidence': 81,
             'Datetime': datetime.datetime(2018, 11, 30, 0, 20, 53),
             'Surface_Temperature_Celcius': 55}}
{'Air_Temperature_Celcius': 15,
 'Hotspot': {'Confidence': 100,
             'Datetime': datetime.datetime(2018, 12, 8, 13, 11, 44),
             'Surface_Temperature_Celcius': 54}}
{'Air_Temperature_Celcius': 15,
 'Hotspot': {'Confidence': 100,
             'Datetime': datetime.datetime(2018, 12, 8, 13, 11, 44),
             'Surface_Temperature_Celcius': 71}}
{'Air_Temperature_Celcius': 15,
 'Hotspot': {'Confidence': 100,
             'Datetime': datetime.datetime(2018, 12, 8, 13, 11, 43),
             'Surface_Temperature_Celcius': 52}}
{'Air_Temperature_Celcius': 17,
 'Hotspot': {'Confidence': 87,
             'Datetime': datetime.datetime(2018, 12, 9, 0, 14, 34),
             'Surface_Temperature_Celcius': 61}}
{'Air_Temperature_Celcius': 17,
 'Hotspot': 

**e.Find the top 10 records with the highest surface temperature (°C).**

In [9]:
search_E = collection.aggregate([    
    {"$unwind": "$Hotspot"},      
    {"$sort" : { "Hotspot.Surface_Temperature_Celcius":-1}}, 
    {"$limit" : 10 },
    {"$project" : {"_id":0, "Hotspot":1}} 
])

for documents in search_E:
    pprint(documents)

{'Hotspot': {'Confidence': 100,
             'Date': datetime.datetime(2018, 4, 18, 0, 0),
             'Datetime': datetime.datetime(2018, 4, 18, 4, 52),
             'Latitude': -38.1665,
             'Longitude': 143.062,
             'Surface_Temperature_Celcius': 124}}
{'Hotspot': {'Confidence': 100,
             'Date': datetime.datetime(2018, 4, 4, 0, 0),
             'Datetime': datetime.datetime(2018, 4, 4, 4, 32, 50),
             'Latitude': -36.343,
             'Longitude': 142.1986,
             'Surface_Temperature_Celcius': 123}}
{'Hotspot': {'Confidence': 100,
             'Date': datetime.datetime(2018, 5, 1, 0, 0),
             'Datetime': datetime.datetime(2018, 5, 1, 4, 14, 20),
             'Latitude': -36.9318,
             'Longitude': 143.0907,
             'Surface_Temperature_Celcius': 122}}
{'Hotspot': {'Confidence': 100,
             'Date': datetime.datetime(2018, 3, 18, 0, 0),
             'Datetime': datetime.datetime(2018, 3, 18, 3, 50, 50),
           

**f.Find the number of fire in each day.**

In [10]:
search_F = collection.aggregate([
        { "$match": {"Hotspot": {"$exists": True}}},
        {"$project": {
            "_id":0,
            "Date": 1,
            "Number_Of_Fires": { "$size": "$Hotspot" }
         }
      }
   ]
)
for documents in search_F:
    if documents['Number_Of_Fires'] != 0:
        pprint(documents)

{'Date': datetime.datetime(2018, 3, 6, 0, 0), 'Number_Of_Fires': 2}
{'Date': datetime.datetime(2018, 3, 7, 0, 0), 'Number_Of_Fires': 1}
{'Date': datetime.datetime(2018, 3, 8, 0, 0), 'Number_Of_Fires': 2}
{'Date': datetime.datetime(2018, 3, 9, 0, 0), 'Number_Of_Fires': 3}
{'Date': datetime.datetime(2018, 3, 10, 0, 0), 'Number_Of_Fires': 8}
{'Date': datetime.datetime(2018, 3, 12, 0, 0), 'Number_Of_Fires': 5}
{'Date': datetime.datetime(2018, 3, 13, 0, 0), 'Number_Of_Fires': 2}
{'Date': datetime.datetime(2018, 3, 14, 0, 0), 'Number_Of_Fires': 10}
{'Date': datetime.datetime(2018, 3, 15, 0, 0), 'Number_Of_Fires': 7}
{'Date': datetime.datetime(2018, 3, 17, 0, 0), 'Number_Of_Fires': 6}
{'Date': datetime.datetime(2018, 3, 18, 0, 0), 'Number_Of_Fires': 3}
{'Date': datetime.datetime(2018, 3, 19, 0, 0), 'Number_Of_Fires': 21}
{'Date': datetime.datetime(2018, 3, 24, 0, 0), 'Number_Of_Fires': 2}
{'Date': datetime.datetime(2018, 3, 25, 0, 0), 'Number_Of_Fires': 13}
{'Date': datetime.datetime(2018, 3,

**g.Find the average surface temperature (°C) for each day.**

In [11]:
search_G = collection.aggregate([
    {"$match": {"Hotspot": {"$exists": True}}},
    {"$project": {
                   "_id":0,
                   "Date": 1,
                   "Average_Surface_Temperature_Celcius":{"$avg":"$Hotspot.Surface_Temperature_Celcius"}}}
    ])

for documents in search_G:
    if documents['Average_Surface_Temperature_Celcius'] != None:
        documents['Average_Surface_Temperature_Celcius'] = round(documents['Average_Surface_Temperature_Celcius'], 2)
        pprint(documents)
        

{'Average_Surface_Temperature_Celcius': 60.5,
 'Date': datetime.datetime(2018, 3, 6, 0, 0)}
{'Average_Surface_Temperature_Celcius': 64.0,
 'Date': datetime.datetime(2018, 3, 7, 0, 0)}
{'Average_Surface_Temperature_Celcius': 51.5,
 'Date': datetime.datetime(2018, 3, 8, 0, 0)}
{'Average_Surface_Temperature_Celcius': 46.67,
 'Date': datetime.datetime(2018, 3, 9, 0, 0)}
{'Average_Surface_Temperature_Celcius': 69.38,
 'Date': datetime.datetime(2018, 3, 10, 0, 0)}
{'Average_Surface_Temperature_Celcius': 88.2,
 'Date': datetime.datetime(2018, 3, 12, 0, 0)}
{'Average_Surface_Temperature_Celcius': 38.5,
 'Date': datetime.datetime(2018, 3, 13, 0, 0)}
{'Average_Surface_Temperature_Celcius': 65.6,
 'Date': datetime.datetime(2018, 3, 14, 0, 0)}
{'Average_Surface_Temperature_Celcius': 46.0,
 'Date': datetime.datetime(2018, 3, 15, 0, 0)}
{'Average_Surface_Temperature_Celcius': 59.5,
 'Date': datetime.datetime(2018, 3, 17, 0, 0)}
{'Average_Surface_Temperature_Celcius': 79.33,
 'Date': datetime.datetim